In [ ]:
import pandas as pd
import numpy as np
import itertools
from Bio import SeqIO
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
import ast

In [ ]:
#input data
##list of GSE studies
gse_id_list = ['GSE145926', 'GSE154244', 'GSE150316']

##min for variant frequency
min_VF = 0.001

##mapping method
method = 'BWA'

##read genome SARS-CoV-2 file
##to find position in genome: str(genome_dict['NC_045512.2'].seq)
genome_file = open('SARS_CoV_2.fasta')
genome_dict = SeqIO.to_dict(SeqIO.parse(genome_file, "fasta"))

#hairpins coordinates in SARS-CoV-2 (positions start with 1)
hairpin_SARS_coordinates = 'hairpin_list.txt'
df_hairpin = pd.read_csv(hairpin_SARS_coordinates, skiprows=1, names=['Loop_length', 'Pos', 'Loop_sequence', 'Stem_length'])
df_hairpin['end_pos'] = df_hairpin['Pos'].apply(lambda x: ast.literal_eval(x)[0][1]-1)

##palette for pictures
colors_list = ['#C6878F', '#81B29A']

In [ ]:
def apobec_mutagenesis(vcf_file, ID, align_method, min_vf=min_VF, genome=str(genome_dict['NC_045512.2'].seq), hairpin_SARS_coordinates=df_hairpin, colors=colors_list):
    
    ##read vcf file
    ##skip description in vcf file
    with open(vcf_file, 'r') as f:
        reader=f.readlines()
        row = 0
        while reader[row].startswith('##') == True:
            row += 1
    df_vcf = pd.read_csv(vcf_file, skiprows=row, sep='\t')
    df_vcf = df_vcf.dropna()
    
    #choose positions with C>T mutation
    df_vcf_C_T = df_vcf[(df_vcf['REF']== 'C') & (df_vcf['ALT']== 'T')]
    df_vcf_C_T.reset_index(inplace=True, drop=True)
    if len(df_vcf_C_T) == 0:
        return 'empty'
    
    #filter by min VF
    FORMAT = df_vcf_C_T.iloc[0, -2].split(':')
    VF_ind = FORMAT.index('VF')
    df_vcf_C_T['VF'] = df_vcf_C_T.iloc[:,-1].apply(lambda x: float(x.split(':')[VF_ind]))
    df_vcf_C_T = df_vcf_C_T[(df_vcf_C_T['VF']>=min_vf)]
    
    df_vcf_C_T['preceding_nucl'] = '-'
    df_vcf_C_T['loop_position'] = '-'
    df_vcf_C_T.reset_index(inplace=True, drop=True)
    for row in range(0, len(df_vcf_C_T)):
        mutation_pos = df_vcf_C_T.loc[row, 'POS'] - 1
        #write letter before mutation C>T
        df_vcf_C_T.at[row, 'preceding_nucl'] = genome[mutation_pos-1]
        
        #write if mutation is in a loop
        end_loop_positions = hairpin_SARS_coordinates.end_pos.to_list()
        if mutation_pos in end_loop_positions:
            df_vcf_C_T.at[row, 'loop_position'] = '+'
     
    
    #make plot
    f, ax = plt.subplots(figsize=(5, 7))
    a = Line2D([], [], color=colors[0], label='-', marker='D', linewidth=1)
    b = Line2D([], [], color=colors[1], label='+', marker='D', linewidth=1)

    sns.stripplot(data=df_vcf_C_T, x="preceding_nucl", y="VF", hue="loop_position", marker='D', size=12, dodge=True,
                  alpha=.2, linewidth=1, jitter=False, palette=colors, hue_order=['-', '+'], order=['A', 'T', 'C', 'G'])
    ax.set(xlabel='Nucleotide before C')
    ax.legend(handles=[a, b], title='Mutation is in a loop')
    f.suptitle('C>T mutations in '+ID+'\naligned using '+align_method)
    plt.grid()
    plt.tight_layout()
    plt.close(f)
    return(f)

In [ ]:
for gse_id in gse_id_list:
    
    ##get list of srr for gse_id
    srr_acc_file_txt = "SRR_Acc_List_"+gse_id+".txt"
    df_srr_acc = pd.read_csv(srr_acc_file_txt, names=['acc'])
    srr_list = df_srr_acc.acc.to_list()
    
    ##path for directory with vcf files for gse_id
    directory = 'bwa/vcf_'+gse_id
    for srr_id in srr_list:
        print(srr_id, end=', ')
        vcf = directory + '/' + srr_id + '_sorted.vcf'
        plot = apobec_mutagenesis(vcf, srr_id, method)
        if plot == 'empty':
            print('no data')
        else:
            plot.savefig(directory+'/pictures/'+srr_id+'_CT_mutations_'+method+'.png', dpi=800)